In [70]:
from bs4 import BeautifulSoup
import qbittorrentapi
import requests

In [71]:
#https://stackoverflow.com/questions/23102833/how-to-scrape-a-website-which-requires-login-using-python-and-beautifulsoup

data = {
    'nev': '<Username>',
    'pass': '<Password>',
}

with requests.Session() as s:
    response = s.post('https://ncore.pro/login.php' , data)
    hitnrun= s.get('https://ncore.pro/hitnrun.php')
    soup = BeautifulSoup(hitnrun.text, 'html.parser')
    torrent_box = soup.find('div',{'class':'box_torrent_all'})
    links = ["https://ncore.pro/"+i['href'] for i in torrent_box.find_all('a')]
    downloadlinks = []
    for link in links:
        response = s.get(link)
        soup = BeautifulSoup(response.text, 'html.parser')
        downloadlinks.append("https://ncore.pro/"+soup.find('div',{'class':'download'}).find('a')['href'])

In [72]:
#https://qbittorrent-api.readthedocs.io/en/latest/
qbt_client = qbittorrentapi.Client(
    host='localhost',
    port='port as an int',
    username='<Username>',
    password='<Username>',
)

try:
    qbt_client.auth_log_in()
except qbittorrentapi.LoginFailed as e:
    print(e)


In [73]:
for torrent in qbt_client.torrents_info():
    if torrent.time_active/60/60 > 168:
        qbt_client.torrents_delete(delete_files=True, torrent_hashes=torrent.hash)

for torrent in downloadlinks:
    qbt_client.torrents_add(urls=torrent,is_sequential_download=True)